# Task 0.2.1

Preparing CIFAR-10 Dataset (Resizing Images to 224x224)

In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize CIFAR-10 images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization values for ImageNet
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

Load and Modify AlexNet

In [12]:
import torch.nn as nn
import torchvision.models as models

# Load pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)

# Modify the classifier to match CIFAR-10 (10 classes)
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=10)
alexnet = alexnet.to('cuda' if torch.cuda.is_available() else 'cpu')


Fine-Tuning (Training the Model from Scratch on CIFAR-10)

In [13]:
import torch
import torch.optim as optim
import torch.nn as nn

# Define the device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
alexnet = alexnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

def train(model, trainloader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader)}")

train(alexnet, trainloader, criterion, optimizer)


Epoch [1/5], Loss: 1.6497810629322707
Epoch [2/5], Loss: 1.2084222049511912
Epoch [3/5], Loss: 1.0414999508491867
Epoch [4/5], Loss: 0.9513980535899892
Epoch [5/5], Loss: 0.8906555700942379


In [14]:
def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")

test(alexnet, testloader)


Test Accuracy: 68.65%


Feature Extraction (Using Pretrained AlexNet)

In [16]:
import torch.optim as optim
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

alexnet = models.alexnet(pretrained=True)
alexnet = alexnet.to(device)

for param in alexnet.parameters():
    param.requires_grad = False

alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.classifier[6].parameters(), lr=0.001)

def train(model, trainloader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader)}")

train(alexnet, trainloader, criterion, optimizer)


Epoch [1/5], Loss: 0.718656501654164
Epoch [2/5], Loss: 0.6343417854412742
Epoch [3/5], Loss: 0.6041440393804284
Epoch [4/5], Loss: 0.5964872168038812
Epoch [5/5], Loss: 0.5897629441278974


In [17]:
def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")

test(alexnet, testloader)

Test Accuracy: 81.33%


# Task 0.2.2

Import Libraries and Load the MNIST Dataset

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

transform = transforms.Compose([
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


Define a CNN Model

In [30]:
class CNN_MNIST(nn.Module):
    def __init__(self):
        super(CNN_MNIST, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128*4*4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 128*4*4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN_MNIST().cuda()


Train the Model on MNIST

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in trainloader:
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {100*correct/total:.2f}%')

torch.save(model.state_dict(), 'mnist_cnn.pth')


Epoch [1/5], Loss: 0.1619, Accuracy: 94.98%
Epoch [2/5], Loss: 0.0422, Accuracy: 98.67%
Epoch [3/5], Loss: 0.0295, Accuracy: 99.07%
Epoch [4/5], Loss: 0.0239, Accuracy: 99.22%
Epoch [5/5], Loss: 0.0193, Accuracy: 99.38%


Evaluate the Model on MNIST Test Set

In [32]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'MNIST Test Accuracy: {100 * correct / total:.2f}%')


MNIST Test Accuracy: 98.96%


Load the SVHN Dataset

In [41]:
svhn_trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
svhn_testset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform)

svhn_trainloader = DataLoader(svhn_trainset, batch_size=64, shuffle=True)
svhn_testloader = DataLoader(svhn_testset, batch_size=64, shuffle=False)


Define the CNN Architecture for SVHN

In [42]:
import torch
import torch.nn as nn

class CNN_SVHN(nn.Module):
    def __init__(self):
        super(CNN_SVHN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128*4*4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 128*4*4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


Load Pre-Trained Model Weights from MNIST

In [47]:
model_svhn = CNN_SVHN().cuda()

model_svhn.load_state_dict(torch.load('mnist_cnn.pth'), strict=False)

for param in model_svhn.parameters():
    param.requires_grad = False

# Unfreeze last convolutional layer conv3
for param in model_svhn.conv3.parameters():
    param.requires_grad = True

#Unfreeze last output layer fc2
for param in model_svhn.fc2.parameters():
    param.requires_grad = True


Feature Extraction on the Model on SVHN, only updating weights of the last fully connected layer.

In [48]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_svhn.fc2.parameters(), lr=0.001)  # Only optimize last layer

num_epochs = 5
for epoch in range(num_epochs):
    model_svhn.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in svhn_trainloader:
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()

        outputs = model_svhn(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(svhn_trainloader):.4f}, Accuracy: {100*correct/total:.2f}%')

torch.save(model_svhn.state_dict(), 'svhn_cnn_finetuned.pth')


Epoch [1/5], Loss: 1.7329, Accuracy: 42.27%
Epoch [2/5], Loss: 1.6226, Accuracy: 45.78%
Epoch [3/5], Loss: 1.5861, Accuracy: 47.08%
Epoch [4/5], Loss: 1.5662, Accuracy: 47.80%
Epoch [5/5], Loss: 1.5531, Accuracy: 48.13%


Evaluate the Fine-Tuned Model on the SVHN Test Set

In [50]:
model_svhn.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in svhn_testloader:
        inputs, labels = inputs.cuda(), labels.cuda()

        outputs = model_svhn(inputs)

        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'SVHN Test Accuracy (with pre-trained CNN): {100 * correct / total:.2f}%')


SVHN Test Accuracy (with pre-trained CNN): 51.01%
